# Import Library & Authenrize

In [0]:
# run this for anthentic
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# for data science
import os
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import re
from string import punctuation
import os
import pandas as pd
import numpy as np
from google.colab import files

from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import nltk

from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
#for file1 in file_list:
#  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))
cdc_list = drive.ListFile({'q': "'144fnA-4uAmg9DXgqwH-cIpr2pGiaa_gk' in parents"}).GetList()
for _file in cdc_list:
  print('title: %s, id: %s, mimeType: %s' % (_file['title'], _file['id'], _file["mimeType"]))
#cdc_list = drive.ListFile({'q': "'1MEFuqJTZQ-ufbvqyIH8wMTiPEA3wxR2w' in parents"}).GetList()
#for _file in cdc_list:
#  print('title: %s, id: %s, mimeType: %s' % (_file['title'], _file['id'], _file["mimeType"]))
# print('connection succeed')

title: NLP for Medical Transcript.ipynb, id: 1JSPF2LKbLpgjwABQOlDjfPbpdiigBKjW, mimeType: application/vnd.google.colaboratory
title: CDC mortality research paper, id: 1Kutx-oqnP7FkP0sxGqffcsNzlhyfwHwad4fI6YxlMao, mimeType: application/vnd.google-apps.document
title: CDC final report, id: 1lnRm3RNyNlrsGRXLQytyKkANMEYa6vfAifNT-s16-S8, mimeType: application/vnd.google-apps.presentation
title: CDC Mortality Research Preliminary Report, id: 1mamC0cekq7MyGdnc_waOnuf6iz-jah3BUxA-O93Kev8, mimeType: application/vnd.google-apps.presentation
title: CDC Mortality.ipynb, id: 1mhTMTw8KStxotzcR0X6e28o2UqcWiPTi, mimeType: application/vnd.google.colaboratory
title: ICD_10th_structured_data, id: 1r16sAMg8snDytAqnMOzY6agENGOAycnl1M0YPaWx2j0, mimeType: application/vnd.google-apps.spreadsheet
title: Data preprocessing.ipynb, id: 1lxoWVX-07xYu3j1TQ0HIMuAjOdZjgkrv, mimeType: application/vnd.google.colaboratory
title: icd_10_structure_complete.csv, id: 1rgwmASDfXvqcwEuH6uqLuT0xA6XeAX-n, mimeType: text/csv
tit

In [0]:
# Unzip the data
# !unzip mortality.zip
dl_med = drive.CreateFile({'id':'1JDM776HTnYsgQiJwL-amWNuXYltuSlmO'}) 
dl_med.GetContentFile('med.csv')

dl_PubMed = drive.CreateFile({'id':'1pNf-L1PoG_gGzrC7VercE3NNy0qFEtYJ'})
dl_PubMed.GetContentFile('PubMed-and-PMC-w2v.bin')

dl_top = drive.CreateFile({'id':'1O1qx9fKjC4hqX80HncNqJejqSC6eBwTg'}) 
dl_top.GetContentFile('Top10_ICD_Description_V2.csv')

mt = pd.read_csv('med.csv')
top10 = pd.read_csv('Top10_ICD_Description_V2.csv')

In [0]:
mt = mt.astype(str)
top10 = top10.astype(str)

In [0]:
mt

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",nan
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",nan
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",nan


In [0]:
top10

,ICD_Code,Top_10_Death,Detailed_Description
0,I251,Atherosclerotic heart disease,Atherosclerotic Coronary (artery): atheroma at...
1,C349,"Malignant neoplasm of bronchus or lung, unspec...","Malignant neoplasm of bronchus or lung, persis..."
2,I219,"Acute myocardial infarction, unspecified","Acute myocardial infarction, unspecified Myoca..."
3,J449,"Chronic obstructive pulmonary disease, unspeci...","Chronic obstructive pulmonary disease, unspeci..."
4,F03,Unspecified dementia,dementia Presenile dementia psychosis Primary ...
5,G309,"Alzheimer's disease, unspecified",Alzheimer disease difficulty remembering newly...
6,I64,"Stroke, not specified as hemorrhage or infarction",Stroke Cerebrovascular accident as haemorrhage...
7,I250,"Atherosclerotic cardiovascular disease, so des...",Atherosclerotic cardiovascular atheroma athero...
8,I500,Congestive heart failure,Congestive heart disease Right ventricular fai...
9,J189,"Pneumonia, unspecified","Pneumonia, unspecified Chest pain when you bre..."


# Method 1 (W2V)

## W2V

### medical transcript W2V

In [0]:
def get_token(text):
  mt_token=[]
  for i in range(len(text)):
      mt_token_pos=nltk.pos_tag(nltk.word_tokenize(text[i]))
      words = [word for word, pos in mt_token_pos if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'
      or pos == 'VB' or pos == 'VBG' or pos == 'VBN' or pos == 'VBD' or pos == 'VBP' or pos == 'VBZ'
      or pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos == 'NNPS' or pos == 'DT')]
      mt_token = mt_token+words
  return mt_token

In [0]:
ds_token = get_token(mt['description'])

KeyboardInterrupt: ignored

In [0]:
  model = KeyedVectors.load_word2vec_format("PubMed-and-PMC-w2v.bin", binary=True)
# Access vectors for specific words with a keyed lookup:

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_row_ave_vectors(text):
  description_vec=np.empty((0, 200), float)

  for i in range(len(text)):
      word2vec_ls=np.empty((0, 200), float)
      mt_token_pos=nltk.pos_tag(nltk.word_tokenize(text[i]))
      for word, pos in mt_token_pos:
          if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'
      or pos == 'VB' or pos == 'VBG' or pos == 'VBN' or pos == 'VBD' or pos == 'VBP' or pos == 'VBZ'
      or pos == 'JJ' or pos == 'JJR' or pos == 'JJS' or pos == 'NNPS' or pos == 'DT'):
              try:
                  word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
              except:
                  pass
      if np.any(np.isnan(word2vec_ls)) or len(word2vec_ls)==0:
          desc_vec=np.zeros((1, 200))
      else:
          desc_vec = np.mean(word2vec_ls, axis=0).reshape(1, 200)
      description_vec=np.vstack([description_vec, desc_vec])
  return description_vec

In [0]:
descrip_W2V = pd.DataFrame(get_row_ave_vectors(mt['description']))
medspec_W2V = pd.DataFrame(get_row_ave_vectors(mt['medical_specialty']))
samplename_W2V = pd.DataFrame(get_row_ave_vectors(mt['sample_name']))
transcrip_W2V = pd.DataFrame(get_row_ave_vectors(mt['transcription']))
keywords_W2V = pd.DataFrame(get_row_ave_vectors(mt['keywords']))

In [0]:
keywords_W2V

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,-0.063085,0.309973,0.008167,-0.167522,-0.112531,0.146547,-0.005166,0.071727,-0.041800,0.127635,-0.029511,-0.224712,0.277056,0.153207,-0.196385,0.027502,-0.093114,-0.039531,0.194356,0.027459,0.147970,0.128924,-0.043685,-0.292560,-0.220644,-0.033683,0.164278,-0.022142,-0.011776,-0.204138,0.169082,0.110241,0.089258,-0.049470,-0.010792,-0.018275,-0.034332,-0.337073,-0.129365,0.081848,...,-0.396361,0.155217,0.101899,0.091587,-0.111479,0.124423,0.020209,0.077204,0.052097,0.111908,0.053413,-0.144635,-0.006416,-0.371336,-0.095542,-0.210204,-0.043953,-0.080933,0.113238,-0.053684,0.195502,0.000159,0.141017,-0.240240,0.126863,0.002512,-0.045232,-0.269245,-0.047028,-0.060789,-0.035039,0.208908,0.032913,-0.309888,0.060375,-0.074076,0.353769,-0.093092,-0.201149,-0.067407
1,-0.057146,0.093366,-0.054909,-0.072129,0.118099,0.042019,0.019381,0.045507,0.042931,0.154448,-0.104803,-0.017489,0.099154,-0.070350,-0.127408,0.068110,0.027628,0.015939,0.147451,0.118623,0.106469,-0.111250,-0.009205,0.015851,-0.073904,0.155837,-0.215318,0.010668,-0.068880,0.104264,0.074702,-0.156232,0.037967,-0.115179,0.061200,0.057465,0.104008,0.008280,-0.030289,-0.272035,...,0.065356,-0.092344,0.012948,0.071571,0.123552,0.241818,0.059970,0.010111,0.172411,0.048077,0.057552,-0.033713,0.118973,-0.090537,-0.172448,0.182258,0.175781,-0.034413,-0.078355,0.060659,0.047204,-0.093255,-0.104957,-0.044557,0.160224,-0.050659,-0.065195,-0.062048,-0.086424,-0.005397,-0.027453,-0.019125,-0.013702,-0.106087,-0.044842,0.020208,-0.080546,0.101568,-0.105279,-0.051307
2,-0.103439,0.091121,-0.005449,-0.138995,0.077849,-0.042706,0.015599,0.100027,0.014241,0.140988,-0.187346,0.008749,0.208341,-0.076120,-0.126747,0.050690,-0.047367,0.055521,0.068728,0.180247,0.110931,-0.000539,-0.017319,0.050034,-0.086630,0.214708,-0.049336,0.065824,-0.117285,0.041751,0.017159,-0.141297,0.037490,-0.142968,0.176428,-0.018063,0.149300,0.012394,-0.010331,-0.192736,...,-0.044911,-0.059672,0.013368,0.027006,0.073685,0.168709,-0.010574,-0.003283,0.115790,0.026922,0.097184,-0.045895,0.079226,-0.022411,-0.231091,0.095528,0.079134,-0.079038,0.073834,-0.030029,0.053464,-0.037686,-0.083639,-0.055996,0.102785,0.051643,-0.047763,-0.044357,-0.114536,-0.035066,0.003321,0.008186,0.030924,-0.150490,-0.134902,0.082040,0.019267,0.081033,-0.081273,-0.084042
3,-0.131607,0.350171,-0.127737,-0.105927,0.151562,-0.123236,0.084608,0.257826,-0.141617,-0.192712,-0.294034,-0.000614,0.599519,-0.259514,-0.116073,-0.147898,-0.254123,0.034375,-0.044888,0.267905,0.364848,0.105875,-0.103789,0.123606,0.089065,0.152300,0.272847,0.053859,-0.275033,0.017058,-0.166114,-0.337420,0.043790,-0.395085,-0.043542,0.024226,-0.064949,-0.013225,-0.025929,-0.009072,...,-0.154287,-0.049985,0.054430,0.080496,0.191045,0.299758,-0.066538,0.023969,-0.197744,0.060133,0.024329,0.032765,0.043375,0.102121,-0.166741,0.035936,0.042291,-0.043340,0.031447,-0.227173,-0.135367,-0.125189,-0.150165,-0.005950,0.050005,0.216760,0.111519,-0.149690,-0.024144,0.096843,-0.015572,-0.099993,0.284800,0.058130,0.035712,0.206855,0.216995,0.057250,-0.015812,-0.137102
4,-0.060510,0.294499,-0.139543,-0.066625,0.193169,-0.066702,0.158758,0.180180,-0.112885,-0.156253,-0.298376,0.033178,0.646498,-0.220240,-0.083946,-0.104122,-0.190920,0.030575,-0.075859,0.215193,0.266268,0.082789,-0.084616,0.136020,0.047044,0.104917,0.298513,0.093186,-0.222351,0.056406,-0.167743,-0.317113,0.046431,-0.315533,0.036339,-0.044869,-0.009381,0.013561,-0.048846,-0.068545,...,-0.124171,-0.048632,0.064846,0.058071,0.125764,0.281176,-0.077095,0.050940,-0.153688,0.056736,0.032374,0.032153,0.088732,0.094592,-0.211502,-0.013004,0.099536,-0.155216,0.088609,-0.205569,-0.062682,-0.104441,-0.149434,-0.032577,0.003507,0.203901,0.070814,-0.073322,-0.038043

### Top 10 ICD code W2V

In [0]:
top10des_W2V = pd.DataFrame(get_row_ave_vectors(top10["Detailed_Description"]))
top10name_W2V = pd.DataFrame(get_row_ave_vectors(top10["Top_10_Death"]))

In [0]:
top10des_W2V

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,-0.043694,0.103156,0.002622,-0.086222,-0.001627,-0.056249,0.000170,0.106771,0.041836,0.139770,-0.085768,-0.068975,0.238368,-0.022744,-0.053717,-0.036628,-0.099086,-0.038496,-0.029942,0.020990,0.159051,0.008685,0.049126,0.038252,-0.003842,-0.020016,0.116601,-0.000404,-0.187175,0.050866,-0.039866,-0.166974,-0.052021,-0.085279,0.117819,0.013418,0.032038,-0.047470,-0.051991,-0.078254,...,-0.148249,0.051764,0.011123,0.086501,-0.022848,0.148876,-0.059402,0.006007,0.094506,0.037122,0.002036,-0.079002,0.066960,-0.061031,-0.203599,-0.024153,0.032838,-0.161498,0.130188,-0.016230,0.088415,0.043122,-0.092645,-0.000567,0.165975,0.013043,-0.007001,0.085947,-0.118917,-0.047210,0.118905,0.033388,-0.006973,-0.031523,-0.029648,0.039405,0.097775,-0.002493,-0.057135,-0.091075
1,-0.103928,0.208812,-0.001502,-0.025019,-0.022648,-0.038726,0.029826,0.036255,0.013898,0.163910,-0.079287,-0.062238,0.222348,-0.056121,-0.120510,0.027166,-0.060524,-0.029164,0.063942,0.026262,0.277287,-0.030105,-0.120238,-0.024287,0.018866,0.032562,0.124933,0.009176,-0.143306,0.073081,-0.055515,-0.064313,-0.046380,-0.095821,0.027964,0.073978,0.023306,-0.088851,-0.115187,-0.027345,...,-0.195953,0.049943,-0.028629,0.012468,0.055224,0.146164,-0.047808,-0.036344,0.018500,-0.011398,-0.057619,-0.106306,0.001264,-0.052672,-0.113398,-0.068400,-0.039958,-0.129271,0.131951,-0.081743,0.084613,-0.008775,-0.038437,0.033131,0.135466,0.050011,-0.041308,-0.034212,-0.082397,-0.052077,0.102207,0.041063,0.011793,-0.106123,-0.035372,-0.005128,0.105490,0.049307,0.054178,-0.083037
2,-0.035405,0.196918,0.045363,-0.050360,-0.034615,-0.026692,-0.012692,0.087784,-0.049626,0.143870,-0.065045,-0.024632,0.224299,-0.085441,-0.101225,-0.069799,-0.141220,-0.047149,0.019566,0.007334,0.201256,0.035428,-0.043274,0.013999,0.010441,-0.037844,0.110821,0.030118,-0.154646,-0.028012,-0.054469,-0.199979,-0.059675,-0.127670,0.125338,0.088698,0.022697,0.030278,-0.084711,-0.058207,...,-0.157947,0.040860,0.028957,0.007570,-0.012252,0.162794,-0.079648,-0.021825,0.083023,0.017364,0.008268,-0.050587,0.065024,-0.022591,-0.166941,-0.015947,-0.003313,-0.151406,0.066519,-0.047195,0.086403,0.036960,-0.053453,0.016058,0.132538,0.055297,0.013903,0.031791,-0.129814,-0.044332,0.100066,-0.034908,-0.041900,-0.023217,-0.044136,0.105964,0.101110,-0.026220,0.027638,-0.071591
3,-0.040086,0.137486,0.059420,-0.096534,0.014788,-0.007779,0.029031,-0.063829,0.027417,0.126432,-0.017595,-0.009333,0.215755,-0.015409,-0.100516,0.005148,-0.023179,-0.072270,0.040682,0.036743,0.290395,0.018067,-0.082927,0.006356,-0.042226,-0.006884,0.132806,-0.034030,-0.156295,-0.051667,0.036856,-0.077123,-0.032698,-0.034191,-0.012938,0.010412,-0.030169,-0.076795,-0.079884,0.011924,...,-0.130791,0.022809,-0.019872,0.015633,0.027502,0.127381,-0.010331,-0.019925,0.046079,0.052362,-0.024608,-0.057215,-0.022597,-0.101367,-0.115133,-0.018302,0.059095,-0.123472,0.071276,-0.034741,0.068059,-0.022583,-0.033524,-0.032544,0.159212,0.097537,0.024118,-0.015127,-0.015022,0.000560,0.030625,-0.008815,0.020855,-0.065075,-0.030794,0.042214,0.112741,0.096115,0.035476,-0.011598
4,-0.199523,0.022527,0.060118,-0.077313,0.095614,-0.054592,-0.002269,0.030365,-0.001955,0.062548,-0.023704,-0.122987,0.183582,-0.052160,0.009455,-0.000474,-0.160265,-0.108263,0.011840,0.083806,0.210870,-0.081393,0.162186,-0.058532,-0.025022,-0.045328,0.061601,-0.129068,-0.196402,0.104029,0.055850,-0.188935,-0.039643,-0.054733,0.072930,-0.021034,-0.000717,-0.042724,-0.030945,-0.035016,...,-0.110941,0.059978,0.014276,0.002795,-0.003711,0.052044,-0.020378,0.024144,0.063289,0.103214,0.020025,-0.148135,0.067394,-0.066001,-0.082114,-0.062215,0.036322,-0.131729,-0.052310,-0.051287,0.005522,0.064022,-0.089088,-0.037261,0.157654,0.016519,-0.009511

## Cosine Similarity

In [0]:
def get_cos_sim(text1, text2):
  return cosine_similarity(text1, text2)


In [0]:
name = get_cos_sim(descrip_W2V, top10name_W2V)
tran = get_cos_sim(transcrip_W2V, top10des_W2V)

In [0]:
name.shape

(4999, 10)

In [0]:
name

array([[0.05194194, 0.28037085, 0.2326758 , ..., 0.10475592, 0.08901195,
        0.25016321],
       [0.19878142, 0.21195207, 0.16587407, ..., 0.17819609, 0.17452776,
        0.05053808],
       [0.19878142, 0.21195207, 0.16587407, ..., 0.17819609, 0.17452776,
        0.05053808],
       ...,
       [0.12194243, 0.18984131, 0.16162883, ..., 0.1833428 , 0.05556855,
        0.22382729],
       [0.11679319, 0.21481775, 0.1922271 , ..., 0.18253901, 0.13111276,
        0.2700647 ],
       [0.34378624, 0.47389111, 0.42159697, ..., 0.37587751, 0.2433435 ,
        0.50526329]])

In [0]:
def select_max(m):
  for i in range(len(m)):
      for j in range(10):
          if m[i][j] != max(m[i]):
              m[i][j] = 0
          else:
              continue
  return m

In [0]:
name_max = select_max(name)
tran_max = select_max(tran)

In [0]:
score = 0.6*name+0.4*tran

In [0]:
score_max = select_max(score)

In [0]:
score

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.3218925 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.3109584 , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.30344806, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.25974315],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
# select samples whose cosine similarity are larger than 0.8
detect_name = np.argwhere(name_max>0.8)
detect_tran = np.argwhere(tran>0.80)

In [0]:
detect_ave = np.argwhere(score>0.7)

In [0]:
detect_ave.shape

(28, 2)

In [0]:
mt.iloc[2979].values

array(['2979',
       ' Patient with a history of coronary artery disease, congestive heart failure, COPD, hypertension, and renal insufficiency.',
       ' Nephrology', ' Renal Insufficiency - Consult ',
       "REASON FOR CONSULT:, Renal insufficiency.,HISTORY OF PRESENT ILLNESS:, A 48-year-old African-American male with a history of coronary artery disease, COPD, congestive heart failure with EF of 20%-25%, hypertension, renal insufficiency, and recurrent episodes of hypertensive emergency, admitted secondary to shortness of breath and productive cough. The patient denies any chest pain, palpitations, syncope, or fever. Denied any urinary disturbances, difficulty, burning micturition, hematuria, or back pain. Nephrology is consulted regarding renal insufficiency.,REVIEW OF SYSTEMS:, Reviewed entirely and negative except for HPI.,PAST MEDICAL HISTORY:, Hypertension, congestive heart failure with ejection fraction of 20%-25% in December 2005, COPD, mild diffuse coronary artery disease

In [0]:
detect_ave

array([[1441,    8],
       [2950,    5],
       [3895,    3],
       [3976,    3],
       [3977,    8],
       [3979,    2],
       [4509,    5],
       [4695,    3],
       [4852,    3],
       [4894,    8],
       [4899,    2]])

In [0]:
detect_name.shape

(8, 2)

In [0]:
detect_tran.shape

(58, 2)

In [0]:
detect_tran

In [0]:
 detect_name.shape

NameError: ignored

In [0]:
detect_tran

array([[   1,    7],
       [1358,    7],
       [1386,    7],
       [1548,    7],
       [1593,    7],
       [1606,    5],
       [1694,    7],
       [1701,    7],
       [1746,    5],
       [1761,    5],
       [1774,    5],
       [1792,    5],
       [1799,    5],
       [2092,    7],
       [2120,    7],
       [2177,    7],
       [2459,    1],
       [2462,    1],
       [2469,    7],
       [2788,    5],
       [2812,    7],
       [2837,    7],
       [2866,    5],
       [2905,    7],
       [2931,    7],
       [2933,    7],
       [2980,    5],
       [3098,    5],
       [3235,    1],
       [3239,    7],
       [3242,    1],
       [3264,    5],
       [3310,    7],
       [3324,    7],
       [3339,    2],
       [3343,    7],
       [3347,    1],
       [3351,    7],
       [3370,    7],
       [3437,    7],
       [4195,    7],
       [4216,    1],
       [4219,    1],
       [4222,    7],
       [4256,    5],
       [4285,    5],
       [4306,    7],
       [4330,

# Method 2 (NLP)

### Medical Transcript

In [0]:
mt.columns

Index(['Unnamed: 0', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords'],
      dtype='object')

In [0]:
train = pd.DataFrame(mt['description'])
train.shape

(4999, 1)

In [0]:
train = train.fillna(" ")

In [0]:
def common_words_transformation_remove_punctuation(text):
    
    text = text.lower()
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"when's", "when is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"there's", "there is", text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\'s", " ", text)  # 除了上面的特殊情况外，“\'s”只能表示所有格，应替换成“ ”
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " america ", text)
    text = re.sub(r" u s ", " america ", text)
    text = re.sub(r" uk ", " england ", text)
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text)
    text = re.sub(r" ds ", " data science ", text)
    text = re.sub(r" ee ", " electronic engineering ", text)
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iphone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the us", "america", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\0s", "0", text)
    
    text = "".join([c for c in text if c not in punctuation])
        
    return text


train['description'] = train['description'].apply(common_words_transformation_remove_punctuation)
train.shape

train.to_csv("train_orig.csv", index = False)

In [0]:
# Download csv
files.download("train_orig.csv")

In [0]:
def remove_stopwords(text):
    stops = set(stopwords.words("english"))
    text = word_tokenize(text)
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

train_stop = train.copy(deep = True)
train_stop['description'] = train_stop['description'].apply(remove_stopwords)
train_stop.shape

(4999, 1)

In [0]:
def stem_words(text):
    text = word_tokenize(text)
    stemmer = SnowballStemmer("english")
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

train_stem = train_stop.copy(deep = True)
train_stem['description'] = train_stem['description'].apply(stem_words)

train_stem.shape

train_stem.to_csv("train_stem.csv", index = False)

In [0]:
def lemmatize_words(text):
    text = word_tokenize(text)
    wordnet_lemmatizer = WordNetLemmatizer()
    lammatized_words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in text]
    text = " ".join(lammatized_words)
    return text

train_lem = train_stop.copy(deep = True)
train_lem['description'] = train_lem['description'].apply(lemmatize_words)

train_lem.shape

train_lem.to_csv("train_lem.csv", index = False)

In [0]:
train_lem

,description
0,23 year old white female present complaint all...
1,consult laparoscopic gastric bypass
2,consult laparoscopic gastric bypass
3,2 mode doppler
4,2 echocardiogram
...,...
4994,patient severe sinusitis two three months ago ...
4995,14 month old baby boy caucasian come presumpti...
4996,female complete physical follow asthma allergi...
4997,mother state wheeze cough


In [0]:
# Download csv
files.download("train_lem.csv")

### ICD code

In [0]:
top10.columns

Index(['Top 10 Death', 'Detailed Description'], dtype='object')

In [0]:
icd = pd.DataFrame(top10['Detailed Description'])
icd.shape

(10, 1)

In [0]:
icd = icd.fillna(" ")

In [0]:
icd['Detailed Description'] = icd['Detailed Description'].apply(common_words_transformation_remove_punctuation)
icd.shape

(10, 1)

In [0]:
icd_stop = icd.copy(deep = True)
icd_stop['Detailed Description'] = icd_stop['Detailed Description'].apply(remove_stopwords)
icd_stop.shape

(10, 1)

In [0]:
icd_lem = icd_stop.copy(deep = True)
icd_lem['Detailed Description'] = icd_lem['Detailed Description'].apply(lemmatize_words)
icd_lem.shape

(10, 1)

In [0]:
icd_lem

,Detailed Description
0,inside artery narrow due build plaque severe r...
1,sign symptoms may suggest lung cancer include ...
2,chest pain common symptom acute myocardial inf...
3,common symptoms copd shortness breath cough pr...
4,dementia broad category brain diseases cause l...
5,disease advance symptoms include problems lang...
6,form stroke associate headache apart subarachn...
7,chest pain pressure usually due enough blood f...
8,leave side heart receive oxygen rich blood lun...
9,people infectious pneumonia often productive c...


## Jaccard Similarity

In [0]:
def shared_words_count(text1, text2):
    wordlist1 = word_tokenize(text1)
    wordlist2 = word_tokenize(text2)
    wordset1 = set(wordlist1)
    wordset2 = set(wordlist2)
    inter = wordset1 & wordset2
    union = wordset1 | wordset2
    count = len(inter)
    #rate = 2 * count / (len(wordset1) + len(wordset2) + 1)  # 为了防止wordset1和wordset2同时为空，也即text1和text2都是只包含一个空格的字符串
    jaccard = count / (len(union) + 1)
    return jaccard

In [0]:
def get_jaccard(text):
  jac = []
  for row in train_lem['description']:
      temp = []
      for r in text:
          temp.append(shared_words_count(r,row))
      jac.append(temp)
  return jac

In [0]:
jaccard = get_jaccard(icd_lem['Detailed Description'])
jaccard_table = pd.DataFrame(data = jaccard, columns=top10['Top 10 Death'].tolist())

In [0]:
jaccard_table.head()

,Atherosclerotic heart disease,"Malignant neoplasm of bronchus or lung, unspecified","Acute myocardial infarction, unspecified","Chronic obstructive pulmonary disease, unspecified",Unspecified dementia,"Alzheimer's disease, unspecified","Stroke, not specified as hemorrhage or infarction",Angina pectoris,Heart failure,"Pneumonia, unspecified"
0,0.046875,0.020833,0.016043,0.014151,0.034483,0.019481,0.020690,0.034884,0.013514,0.022901
1,0.050000,0.021429,0.016304,0.014286,0.035294,0.020000,0.021127,0.036585,0.013699,0.023256
2,0.050000,0.021429,0.016304,0.014286,0.035294,0.020000,0.021127,0.036585,0.013699,0.023256
3,0.050847,0.021583,0.016393,0.014354,0.035714,0.020134,0.021277,0.037037,0.013761,0.023622
4,0.051724,0.021739,0.016484,0.014423,0.036145,0.020270,0.021429,0.037500,0.013825,0.023810
...,...,...,...,...,...,...,...,...,...,...
4994,0.042254,0.020000,0.015464,0.013825,0.031915,0.018634,0.019737,0.032967,0.013158,0.021898
4995,0.037975,0.018987,0.014778,0.013100,0.028846,0.017857,0.018634,0.029412,0.012605,0.020408
4996,0.048387,0.020979,0.016043,0.014151,0.034091,0.019608,0.020833,0.035294,0.013636,0.022727
4997,0.050000,0.021739,0.016304,0.014423,0.035294,0.020000,0.021127,0.036585,0.013761,0.023622


In [0]:
jaccard_table.iloc[1441,:]

Atherosclerotic heart disease                          0.046875
Malignant neoplasm of bronchus or lung, unspecified    0.020833
Acute myocardial infarction, unspecified               0.016129
Chronic obstructive pulmonary disease, unspecified     0.014151
Unspecified dementia                                   0.033333
Alzheimer's disease, unspecified                       0.019355
Stroke, not specified as hemorrhage or infarction      0.020408
Angina pectoris                                        0.035294
Heart failure                                          0.013636
Pneumonia, unspecified                                 0.022388
Name: 1441, dtype: float64